<a href="https://colab.research.google.com/github/cphoward/handson-ml3/blob/main/ch3-exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', as_frame=False)

In [9]:
X, y = mnist.data, mnist.target

In [10]:
X.shape

(70000, 784)

In [11]:
y.shape

(70000,)

In [12]:
y[5]

'2'

In [13]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

In [17]:
from sklearn.neighbors import KNeighborsClassifier

In [16]:
knn_cls = KNeighborsClassifier()
knn_cls.fit(X_train, y_train)
baseline_accuracy = knn_cls.score(X_test, y_test)
baseline_accuracy

0.9688

In [21]:
# Suggest imports and settings to pass to HalvingRandomSearchCV
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

from scipy.stats import randint

param_distributions = {
    "weights": ["uniform", "distance"],
    "n_neighbors": randint(low=1, high=10)
}

In [22]:
# Let's fine-tune it
knn_clf = KNeighborsClassifier()

search = HalvingRandomSearchCV(
    estimator=knn_clf,
    param_distributions=param_distributions,
    cv=5,
    # number of initial random candidates; adjust to your appetite for exploration
    n_candidates=40,
    factor=3,                  # how aggressively we halve (3≈“keep ~1/3 each round”)
    resource="n_samples",      # grow the number of training samples each round (default)
    max_resources="auto",      # use all provided samples at the final round
    aggressive_elimination=False,
    random_state=42,           # reproducibility
    return_train_score=True,
)

search.fit(X_train, y_train)
print(search.best_params_)
print(search.best_score_)

{'n_neighbors': 4, 'weights': 'distance'}
0.9114814814814813


In [23]:
search.best_estimator_.fit(X_train, y_train)
tuned_accuracy = search.best_estimator_.score(X_test, y_test)
tuned_accuracy

0.9714